In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import sklearn
from sklearn import datasets

In [2]:
data = sklearn.datasets.load_diabetes(return_X_y=False)
X = data['data']
print(X.shape)
y = data['target'].reshape(-1, 1)
print(y.shape)

(442, 10)
(442, 1)


In [3]:
def toNpArray(X, y, varnames):
    n, p = X.shape
    if isinstance(X, pd.DataFrame):
        if varnames is None and list(X.columns) != list(range(X.shape[1])):
            varnames = np.array(X.columns).reshape(-1,1)
        X = np.array(X)
    elif isinstance(X, list):
        X = np.array(X)
        
    if isinstance(y, pd.Series):
        y = np.array(y).reshape(-1,1)
    elif isinstance(y, list):
        y = np.array(y).reshape(-1,1)
        
    return X, y, varnames

In [4]:
# function sample beta0 from the posterior distribution
# validated
def sample_beta0(X, z, mu_z, Xt1, b, sigma2, omega2):
    if mu_z is not None:
        n = len(z)
        if Xt1 is None:
            m = mu_z
        else:
            m = (mu_z - np.matmul(b.T, Xt1)/n)[0][0]
        v = sigma2/n
    else:
        W = np.sum(1/omega2)
        m = np.sum((z - np.matmul(X, b))/omega2) / W
        v = sigma2 / W
    
    b0 = m + np.random.normal()*np.sqrt(v)
    return b0, m

In [5]:
# function generate random variables from the inverse normal distribution
# validated
def randinvg(mu, lamda):
    lamda = 1/lamda
    p = len(mu)
    V = np.random.normal(size = (p, 1))**2
    
    out = mu + 0.5*mu/lamda*(mu*V - np.sqrt(4*mu*lamda*V+mu**2*V**2))
    
    l = (np.random.uniform(size = (p, 1)) >= mu/(mu + out))
    out[l] = mu[l]**2/out[l]
    
    return out

In [6]:
# function generate random variables from exponential distribution
# validated
def exprnd_fast(mu):
    if type(mu) == type(1.0):
        n = 1
    else:
        n = len(mu)
    r = -mu * np.log(np.random.uniform(size = (n, 1)))
    return r

In [7]:
# function which standardise the design matrix X and target y if required
# validated
def standardise(X, y):
    n = X.shape[0]
    meanX = np.mean(X, axis=0)
    stdX = np.std(X, axis=0)*np.sqrt(n)
    
    X = X - meanX
    X = X/stdX
    
    if y is not None:
        meany = np.mean(y)
        y = y - meany
        return X, meanX, stdX, y, meany
    else:
        return X, meanX, stdX

In [8]:
# function which reverse the standardisation process
# validated
def unstandardise(X, muX, normX, y, muy):
    X = X*normX + muX
    if y is not None:
        y = y + muy
    return X, y

In [9]:
# function forms the diagonal entries of the regularization matrix
# validated
def make_Lambda(sigma2, tau2, lambda2):
    delta2prod = np.ones(shape=(len(lambda2), 1))
    Lambda = sigma2 * tau2 * lambda2 * delta2prod;
    return Lambda, delta2prod

In [10]:
# function sample beta from the posterior distribution
# validated
def sample_beta(X, z, mvnrue, b0, sigma2, tau2, lambda2, delta2prod, omega2, XtX, Xty, Xt1, weights, gprior, b):
    sigma = np.sqrt(sigma2)
    Lambda = sigma2 * tau2 * lambda2 * delta2prod
    
    # no block sampling at this time
    if weights or Xty is None or not mvnrue:
        alpha = z - b0
        
    if mvnrue:
        if Xty is None or weights:
            omega = np.sqrt(omega2)
            X = X/omega
            b, muB = fastmvg_rue(X, None, alpha, None, Lambda, sigma2, omega, gprior, XtX)
        else:
            if Xt1 is None:
                b, muB = fastmvg_rue(None, XtX, None, Xty, Lambda, sigma2, None, gprior, XtX)
            else:
                b, muB = fastmvg_rue(None, XtX, None, Xty - b0*Xt1, Lambda, sigma2, None, gprior, XtX)
    else:
        omega = np.sqrt(omega2)
        X = X/omega
        b, muB = fastmvg_bhat(X, alpha, Lambda, sigma, omega)
        
    return b, muB

In [11]:
# sampler for multivariate gaussian
# validated
def fastmvg_bhat(Phi, alpha, D, sigma, omega):
    n, p = Phi.shape
    u = np.random.normal(size = (p, 1)) * np.sqrt(D)
    delta = np.random.normal(size = (n, 1))
    
    v = np.matmul(Phi, u)/sigma + delta
    Dpt = Phi.T*D/sigma
    W = np.matmul(Phi, Dpt)/sigma + np.eye(n)
    w = np.linalg.solve(W, (alpha/omega/sigma - v))
    x = u + np.matmul(Dpt, w)
    u = x
    return x, u

In [12]:
# sampler for multivariate gaussian
# validated
def fastmvg_rue(Phi, PtP, alpha, Ptalpha, D, sigma2, omega, gprior, XtX):
    '''
    Phi: n x p
    PtP: p x p
    alpha: n x 1
    Ptalpha: p x 1
    D: p x 1
    sigma2: 1 x 1
    omega: n x 1
    gprior: 1 x 1
    XtX: p x p
    '''
    if PtP is None:
        PtP = np.matmul(Phi.T, Phi)
        
    if Ptalpha is None:
        Ptalpha = np.matmul(Phi.T, (alpha/omega))
        
    p = len(D)
    
    if not gprior:
        L = np.linalg.cholesky(PtP/sigma2 + np.diag(1/D.reshape(-1)))
    else:
        L = np.linalg.cholesky(PtP/sigma2 + XtX/D[0])
    
    v = np.linalg.solve(L, Ptalpha/sigma2)
    m = np.linalg.solve(L.T, v)
    w = np.linalg.solve(L.T, np.random.normal(size = (p, 1)))
    x = m + w
    return x, m

In [13]:
# function sample error variance for linear regression model
# validated
def sample_sigma2(mu, y, b, ete, omega2, tau2, lambda2, delta2prod, gprior):
    '''
    mu: n x 1
    y: n x 1
    b: p x 1
    ete: 1 x 1
    omega2: n x 1
    tau2: 1 x 1
    lambda2: p x 1
    delta2prod: p x 1
    gprior: 1 x 1
    '''
    n = len(y)
    p = len(b)
    
    e = None
    if ete is None:
        e = y - mu
        ete = np.sum(e**2/omega2, axis = 0)
    
    shape = (n+p)/2
    
    if not gprior:
        scale = ete/2 + np.sum(b**2/lambda2/delta2prod, axis=0)/2/tau2
    else:
        bXtXb = np.matmul(mu.T, mu)
        scale = np.sum(e**2/omega2, axis=0)/2 + bXtXb/tau2/2
    sigma2 = scale/np.random.gamma(shape)
    muSigma2 = scale/(shape-1)
    return sigma2, muSigma2, e#, mu

In [14]:
# function sample global variance hyperparameter
# validated
def sample_tau2(b, sigma2, lambda2, delta2prod, xi, mu, gprior, tau_a):
    '''
    b: p x 1
    sigma2: 1 x 1
    lambda2: p x 1
    delta2prod: p x 1
    xi: 1 x 1
    mu: n x 1
    gprior: 1 x 1
    tau_a: 1 x 1
    '''
    p = len(b)
    shape = p/2 + tau_a
    
    if not gprior:
        scale = 1/xi + np.sum(b**2/lambda2/delta2prod, axis=0)/2/sigma2
    else:
        scale = 1/xi + np.matmul(mu.T, mu)/2/sigma2
    tau2 = scale/ np.random.gamma(shape)
    muTau2 = scale/(shape-1)
    return tau2, muTau2

In [15]:
# function sample xi for all models
# validated
def sample_xi(tau2, tau_ab):
    scale = 1 + 1/tau2
    if tau_ab == 1:
        xi = 1/exprnd_fast(1/scale)
    else:
        shape = tau_ab
        xi = scale/np.random.gamma(shape)
    return xi

In [16]:
# sample lambda2 for LASSO
# validated
def sample_lambda2_lasso(b, sigma2, tau2, delta2prod):
    mu = np.sqrt(2*tau2*sigma2*delta2prod/b**2)
    shape = 2
    lambda2 = 1/randinvg(mu, 1/shape)
    return lambda2

In [17]:
# function computes the probability of data for the linear models
# validated
def br_regnlike_mu(error, mu, e, y, s2, tdof):
    '''
    error: error model, gaussian, laplace, t
    mu: n x 1
    e: n x 1
    y: n x 1
    s2: 1 x 1
    tdof: 1 x 1 
    '''
    
    if error != 'binomial':
        if error == 'gaussian':
            neglogprob = e**2/s2/2 + (1/2)*np.log(2*np.pi*s2)
        elif error == 'laplace':
            scale = np.sqrt(s2/2)
            neglogprob = np.abs(e)/scale + np.log(2*scale)
        elif error == 't':
            nu = tdof
            neglogprob = -sp.special.gammaln((nu+1)/2) + sp.special.gammaln(nu/2) + (nu+1)/2*np.log(1+1/nu*e**2/s2) + np.log(np.pi*nu*s2/2)
            
    prob = np.exp(-neglogprob)
    neglike = np.sum(neglogprob)
    
    return neglike, neglogprob, prob

In [18]:
# function calculate the effective sample size
# validated
def ess(x):
    n = len(x)
    s = min(n-1, 2000)
    g = my_autocorr(x, s)
    
    G = g[1:s] + g[2:s+1]
    ix = [i for i, x in enumerate((G < 0).reshape(-1)) if x]
    
    ESS = 0
    ESSfrac = 0
    if len(ix) > 0:
        k = ix[0]
        
        V = g[0] + 2*np.sum(g[1:k+1])
        ACT = V/g[0]
        ESS = min(n/ACT, n)
        ESSfrac = ESS/n
    
    return ESS, ESSfrac

In [19]:
# function calculate autocorrelation
# validated
def my_autocorr(y, order):
    y = y - np.mean(y)
    nFFT = int(2**(nextpow2(len(y)) + 1))
    F = np.fft.fft(y, nFFT, axis=0)
    F = F * np.conj(F)
    acf = np.fft.ifft(F, axis=0)
    
    acf = acf[:order+1]
    acf = np.real(acf)
    acf = acf/acf[0]
    return acf

In [20]:
# function get the smallest number 2 to the power of greater than n
# validated
def nextpow2(n):
    n = abs(n)
    if n == 0:
        return 0.0
    else:
        return np.ceil(np.log2(n))

In [21]:
# function compute the probability of data for the regression models
# validated
def br_regnlike(error, X, y, beta, beta0, s2, tdof):
    mu = np.matmul(X, beta) + beta0
    
    if error != 'binomial':
        e = mu - y
        
        if error == 'gaussian':
            neglogprob = e**2/s2/2 + (1/2)*np.log(2*np.pi*s2)
        elif error == 'laplace':
            scale = np.sqrt(s2/2)
            neglogprob = np.abs(e)/scale + np.log(2*scale)
        elif error == 't':
            nu = tdof
            neglogprob = -sp.special.gammaln((nu+1)/2) + sp.special.gammaln(nu/2) + (nu+1)/2*np.log(1+1/nu*e**2/s2) + np.log(np.pi*nu*s2/2)
            
    prob = np.exp(-neglogprob)
    neglike = np.sum(neglogprob)
    
    return neglike, neglogprob, prob, mu

In [22]:
# function sample the lambda2 for horseshoe prior
# validated
def sample_lambda2_hs(b, sigma2, tau2, nu, delta2prod):
    scale = 1/nu + b**2/2/tau2/sigma2/delta2prod
    lambda2 = 1/exprnd_fast(1/scale)
    return lambda2

In [23]:
# function sample nu for horseshoe prior
# validated
def sample_nu_hs(lambda2):
    scale = 1 + 1/lambda2
    nu = 1/exprnd_fast(1/scale)
    return nu

In [24]:
# function calculate the percentile
# validated
def prctile(X, perct):
    '''
    X: n x p
    perct: numeric
    '''
    N, _ = X.shape
    X_sort = np.sort(X, axis = 0)
    
    perct_array = np.array([100*((0.5 + n)/N) for n in range(N)])
    if perct in perct_array:
        return X_sort[perct_array == perct][0].reshape(-1, 1)
    else:
        if perct < perct_array[0]:
            return X_sort[0][0].reshape(-1, 1)
        elif perct > perct_array[-1]:
            return X_sort[-1][0].reshape(-1, 1)
        else:
            X_low = X_sort[perct_array < perct][-1]
            X_up = X_sort[perct_array > perct][0]
            p_low = perct_array[perct_array < perct][-1]
            p_up = perct_array[perct_array > perct][0]
            return (X_low + ((perct - p_low)/(p_up - p_low))*(X_up - X_low)).reshape(-1, 1)

In [25]:
# function get the ranking of variables
# validated
def bfr(b):
    '''
    Inputs:
        b: [p x nsamples] Regression parameters
    
    Return:
        varranks: [p x 1]
    '''
    p, nsamples = b.shape

    ranks = np.zeros(shape = (p, nsamples))
    
    for i in range(nsamples):
        value = np.abs(b[:,i])
        O = [x for _, x in sorted(zip(value, list(range(len(value)))), reverse=True)]
        ranks[O, i] = range(1, p+1)
        
    q = prctile(ranks.T, 75)
    O = [x for _, x in sorted(zip(q, list(range(len(q)))))]

    varranks = np.array([[None] for i in range(p+1)])
    j = 1
    k = 1
    
    for i in range(p):
        if i >= 1:
            if q[O[i]] != q[O[i-1]]:
                j += k
                k = 1
            else:
                k += 1
        varranks[O[i]] = j

    return varranks

In [26]:
# function compute the model stats
# validated
def br_compute_model_stats(y, X, retval):
    # model type
    gaussian = False
    lapalce = False
    tdist = False
    binomial = False

    if retval['runstats']['model'] in ['binomial','logistic']:
        binomial = True
        model = 'binomial'
    elif retval['runstats']['model'] in ['gaussian','normal']:
        gaussian = True
        model = 'gaussian'
    elif retval['runstats']['model'] in ['laplace', 'l1']:
        laplace = True
        model = 'laplace'
    elif retval['runstats']['model'] in ['t','student']:
        tdist = True
        model = 't'
        
    # stats for continuous model
    if not binomial:
        mu = np.matmul(X, retval['muB']) + retval['muB0']
        modelstats = {}
        
        modelstats['logl'] = -br_regnlike(model, X, y, retval['muB'], retval['muB0'], retval['muSigma2'], retval['runstats']['tdof'])[0]
        modelstats['r2'] = 1 - np.sum((y - mu)**2) / np.sum((y - np.mean(y))**2)
        
    return modelstats

In [30]:
def br_summary(beta, beta0, retval):
    varnames = retval['Xstats']['varnames']
    nx = retval['Xstats']['nx']
    px = retval['Xstats']['px']
    
    model = retval['runstats']['model']
    prior = retval['runstats']['prior']
    nsamples = retval['runstats']['nsamples']
    burnin = retval['runstats']['burnin']
    thin = retval['runstats']['thin']
    normalize = retval['runstats']['normalize']
    runBFR = retval['runstats']['runBFR']
    sortrank = retval['runstats']['sortrank']
    displayor = retval['runstats']['displayor']
    tdof = retval['runstats']['tdof']
    #isVarCat = retval['runstats']['']
    #XtoZ = retval['runstats']['']
    
    # Model type
    gaussian = False
    lapalce = False
    tdist = False
    binomial = False

    if model in ['binomial','logistic']:
        binomial = True
        model = 'binomial'
    elif model in ['gaussian','normal']:
        gaussian = True
        model = 'gaussian'
    elif model in ['laplace', 'l1']:
        laplace = True
        model = 'laplace'
    elif model in ['t','student']:
        tdist = True
        model = 't'
        
    # compute ess for each variable
    ESSfrac = np.zeros(shape = (px, 1))
    for j in range(px):
        _, ESSfrac[j] = ess(beta[j,:])
    
    # table symbols
    chline = '-'
    cvline = '|'
    cTT    = '+'
    
    # find length of the longest variable name
    maxlen = 12
    for i in range(px):
        if len(varnames[i]) > maxlen:
            maxlen = len(varnames[i])
    
    fmtstr = ' ' * maxlen
    
    # display pretable information
    if binomial:
        modeltxt = 'logistic'
    else:
        modeltxt = 'linear'
    
    excess_s = ' ' * (maxlen - 12)
    print('Bayesian ' + modeltxt + ' ' + prior + ' regression', '\n\n')
    print('Number of obs   =' + excess_s + str(nx))
    print('Number of vars  =' + excess_s + str(px),'\n')
    
    if not binomial:
        s2 = retval['muSigma2']
        if tdist:
            s2 = tdof/(tdof-2)*s2
        print('MCMC Samples    =' + excess_s + str(nsamples))
        print('MCMC Burnin     =' + excess_s + str(burnin))
        print('MCMC Thining    =' + excess_s + str(thin), '\n')
        
        print('Root MSE        =' + excess_s + str(np.sqrt(s2)))
        print('R-Squared       =' + excess_s + str(retval['modelstats']['r2']))
        print('WAIC            =' + excess_s + str(retval['modelstats']['waic']), '\n')
        
    # print table header
    print(chline*(maxlen+1), cTT, chline*83)
    print(' '*(maxlen - 10) + 'Parameter'+' '*6+'mean(Coef)    std(Coef)        [95% Cred. Interval]      tStat    Rank       ESS')
    print(chline*(maxlen+1), cTT, chline*83)
    
    # variable information
    if runBFR and sortrank:
        indices = [x for _, x in sorted(zip(retval['varranks'], list(range(len(retval['varranks'])))))]
    else:
        indices = list(range(px+1))
        
    incat = -1
    for i in range(px+1):
        k = indices[i]
        
        # regression variables
        if k < px:
            kappa = retval['tStats'][k]
            s = beta[k,:]
            mu = retval['muB'][k]
            if binomial:
                mu = retval['medB'][k]
        
        # intercept
        elif k == px+1:
            s = beta0
            mu = np.mean(s)
            if binomial:
                mu = retval['medB0']
        
        # compute the credible intervals
        std_err = np.std(s)
        s = s.reshape(-1,1)
        qlin = np.concatenate((prctile(s, 2.5).reshape(1, -1), prctile(s, 25).reshape(1, -1), prctile(s, 75).reshape(1, -1), prctile(s, 97.5).reshape(1, -1)), axis=0)
        qlog = np.concatenate((prctile(np.exp(s), 2.5).reshape(1, -1), prctile(np.exp(s), 25).reshape(1, -1), prctile(np.exp(s), 75).reshape(1, -1), prctile(np.exp(s), 97.5).reshape(1, -1)), axis=0)
        
        q = qlin
        if binomial and displayor:
            mu = np.exp(mu)
            std_err=  (qlog[-1] - qlog[0])/2/1.96
            q = qlog
        
        #display results
        if k > px:
            tstats = '        .'
        else:
            tstats = '    '+str(kappa)
        if retval['varranks'][k] is None:
            rank = '        .'
        else:
            rank = '    ' + str(retval['varranks'][k])
        print(maxlen*' ' + cvline +'    ' +str(mu)+'    '+str(std_err)+'    '+str(q[0])+'    '+str(q[-1])+tstats+rank)

In [32]:
def bayesreg(X, y, model, prior, normalize = True, runBFR = True, nsamples = 1000, burnin = 1000, thin = 5, display = True, 
             displayor = False, varnames = None, sortrank = False, tdof = 5, catvars = None, nogrouping = False, 
             groups = None, tau2prior = [0.5,0.5], blocksample = None, blocksize = None, waic = True):
    '''
    Input parameters:
        Required:
            X           - [n x p], design matrix

            y           - [n x 1], target vector

            model       - str, population distribution

            prior       - str, prior distribution
            
        Optional:
            normalize   - bool, whether to normalize design matrix X, default True
            
            runBFR      - bool, whether to compute ranking of importance for predictors, default: True
            
            nsamples    - int, number of posterior samples, default: 1000

            burnin      - int, number of burnin samples, default: 1000

            thin        - int, level of thining, default: 5

            display     - bool, whether to display summary stats, default: True

            displayor   - bool, whether to display odds ratio for logistic regression, default: False

            varnames    - [p x 1]/None, names for the variables, default: None

            sortrank    - bool, whether to display variables as the order of ranking, default: False

            tdof        - int, degree of freedom for student-t distribution, default: 5

            carvars     - [ncat x 1]/None indexes of categorical variables, default: None

            nogrouping  - bool, whether stop automatically grouping categorical variables after expansion, default: False

            groups      - None/2D list with the first dimension indicating groups and the second dimension indicating which predictors are in the group, default: None

            tau2prior   - str/[a, b], hyperparameters of beta prior on shrinkage parameters, default: [0.5, 0.5]

            blocksample - None/int, the number of blocks when sampling beta, default: None

            blocksize   - None/int, the approximate size of each block when sampling beta, default: None

            waic        - bool, whether to compute WAIC, default: True
    
    Return parameters:
            beta        - [p x nsamples], posterior samples of beta
            beta0       - [1 x nsamples], posterior samples of beta0
            retval      - dict, additional sampling information
    '''
    
    # format X, y to numpy array
    X, y, varnames = toNpArray(X, y, varnames)
    
    # Data dimension
    nx, px = X.shape
    ny, py = y.shape
    
    # Constants
    MAX_PRECOMPUTED_PX = 2e4
    
    #expectedModel= ['gaussian', 'normal', 'laplace', 't', 'studentt', 'binomial', 'logistic']
    #expectedPrior = ['ridge','rr','horseshoe','hs','lasso','hs+','horseshoe+','gprior','g']
    #expectedTau2Prior = ['hc','sb','uniform']
    
    # Model type
    gaussian = False
    laplace = False
    tdist = False
    binomial = False
    
    if model in ["binomial", "logistic"]:
        binomial = True
        model = "binomial"
    elif model in ["gaussian", "normal"]:
        gaussian = True
        model = "gaussian"
    elif model in ["laplace", "l1"]:
        laplace = True
        model = "laplace"
    elif model in ["t", "student"]:
        tdist = True
        model = "t"
        
    # Prior type
    gprior = False
    ridge = False
    lasso = False
    horseshoe = False
    horseshoeplus = False
    
    if prior in ["gprior", "g"]:
        gprior = True
        prior = 'g'
        nogrouping = True
    elif prior in ["ridge", "rr"]:
        ridge = True
        prior = 'ridge'
        nogrouping = True
    elif prior in ["lasso"]:
        lasso = True
        prior = 'lasso'
    elif prior in ["horseshoe", "hs"]:
        horseshoe = True
        prior = 'horseshoe'
    elif prior in ["horseshoe+", "hs+"]:
        horseshoeplus = True
        prior = 'horseshoe+'
        
    # type of tau2 prior
    if isinstance(tau2prior, str):
        if tau2prior == 'hc':
            tau_a = 0.5
            tau_b = 0.5
        elif tau2prior == 'sb':
            tau_a = 0.5
            tau_b = 1.0
        elif tau2prior == 'uniform':
            tau_a = 1.0
            tau_b = 1.0
    else:
        tau_a = tau2prior[0]
        tau_b = tau2prior[1]
    
    # create temporary variable names
    if varnames is None:
        varnames = np.array([['v'+str(i+1)] for i in range(px)])
        np.concatenate((varnames, [['cons']]), axis=0)
        
    # setup variable processing rule
    vars_ = {}
    vars_['description'] = 'Variable information'
    vars_['varnames'] = varnames
    vars_['isVarCat'] = np.array([[False] for i in range(px)])
    vars_['isVarCat'][catvars] = True
    
    
    # change y to z   
    z = y
    weights = laplace or tdist or binomial
    
    # normalize data
    if not normalize:
        muX = np.zeros(shape=(1, px))
        normX = np.ones(shape=(1, px))
    else:
        X, muX, normX = standardise(X, None)
        
    # return values
    retval = {"sparsify_method":"", 'sparseB0':None, "sparseB": None, "medB0": None, "medB": None,
              "muB0": 0, "muB": np.zeros(shape=(px, 1)), "tau2": np.zeros(shape=(1, nsamples)), "xi": np.zeros(shape=(1, nsamples))}
    beta0 = np.zeros(shape = (1, nsamples))
    beta = np.zeros(shape = (px, nsamples))
    
    if not binomial:
        retval["sigma2"] = np.zeros(shape=(1, nsamples))
        retval["muSigma2"] = 0
    if not (ridge or gprior):
        retval["lambda2"] = np.zeros(shape = (px, nsamples))
    
    # Initial values for sampling
    b = np.random.normal(size=(px, 1))
    sigma2 = 1
    e = None
    tau2 = 1
    xi = 1
    lambda2 = np.ones(shape=(px, 1))
    omega2 = np.ones(shape=(nx, 1))
    nu = np.ones(shape=(px, 1))
    phi2 = np.ones(shape=(px, 1))
    zeta = np.ones(shape=(px, 1))
    XtX = None
    Xty = None
    Xt1 = None
    negll = np.zeros(shape=(1, nsamples))
    waicProb = np.zeros(shape=(nx, 1))
    waicLProb = np.zeros(shape=(nx, 1))
    waicLProb2 = np.zeros(shape=(nx, 1))
    
    # determine sampling algorithm
    mvnrue = True
    if(px/nx >= 2):
        mvnrue = False;
    
    # precomputation
    precompute = False
    if ((gaussian and mvnrue) or gprior) and px < MAX_PRECOMPUTED_PX:
        precompute = True
        if gaussian:
            yty = np.matmul(z.T, z)
            Xty = np.matmul(X.T, z)
        XtX = np.matmul(X.T, X)
    
    #Always precompute mean(z) if Gaussian
    mu_z = None
    if gaussian:
        mu_z = np.mean(z)
        if not normalize:
            Xt1 = np.sum(X, axis=1).reshape(-1,1)
            
    # Statistics for result structure retval
    retval["runstats"] = {}
    retval["runstats"]["description"] = "run arguments"
    retval['runstats']['model']= model
    retval['runstats']['prior'] = prior
    retval['runstats']['nsamples'] = nsamples
    retval['runstats']['burnin'] = burnin
    retval['runstats']['thin'] = thin
    retval['runstats']['normalize'] = normalize
    retval['runstats']['runBFR']= runBFR
    retval['runstats']['sortrank'] = sortrank
    retval['runstats']['displayor'] = displayor
    retval['runstats']['blocksample'] = blocksize
    retval['runstats']['tdof'] = tdof
    retval['runstats']['tau2prior'] = [tau_a, tau_b]
    
    if(~tdist):
        retval['runstats']['tdof'] = None
        
    # X stats
    retval["Xstats"] = {}
    retval['Xstats']['description'] = 'Predictor matrix statistics'
    retval['Xstats']['varnames'] = varnames
    retval['Xstats']['nx'] = nx
    retval['Xstats']['px'] = px
    retval['Xstats']['muX'] = muX
    retval['Xstats']['normX'] = normX
    
    ######################################
    k = -1
    iters = 0
    while k < nsamples-1:
        # sample beta0
        b0, muB0 = sample_beta0(X, z, mu_z, Xt1, b, sigma2, omega2)
        
        # form diagonal Lambda matrix
        _, delta2prod = make_Lambda(sigma2, tau2, lambda2)
        
        # sample beta
        b, muB = sample_beta(X, z, mvnrue, b0, sigma2, tau2, lambda2, delta2prod, omega2, XtX, Xty, Xt1, weights, gprior, b)
        
        mu = None
        if gprior or XtX is None or nx < px or waic:
            mu = np.matmul(X, b) + b0
        
        # sample sigma2
        if not binomial:
            ete = None
            if mu is None:
                if Xt1 is None:
                    ete = yty - 2*np.matmul(Xty.T, b) + np.matmul(np.matmul(b.T, XtX), b) + b0*2*nx - 2*mu_z*nx*b0
                else:
                    ete = yty - 2*np.matmul(Xty.T, b) + np.matmul(np.matmul(np.concatenate((b, np.array([[b0]]))).T, np.concatenate((np.concatenate((XtX, Xt1)), np.concatenate((Xt1.T, np.array([[nx]])))), axis=1)), np.concatenate(b, np.array([[b0]]))) - 2*mu_z*nx*b0
            sigma2, muSigma2, e = sample_sigma2(mu, y, b, ete, omega2, tau2, lambda2, delta2prod, gprior)
        
        # sample omega2
        if weights:
            if laplace:
                omega2 = sample_omega2_laplace(e, sigma2)
            elif tdist:
                omega2 = sample_omega2_tdist(e, sigma2, tdof)
        
        # sample tau2
        tau2, _ = sample_tau2(b, sigma2, lambda2, delta2prod, xi, mu, gprior, tau_a)
        
        # sample xi
        xi = sample_xi(tau2, tau_a + tau_b)
        
        # individual shrinkage
        if lasso:
            lambda2 = sample_lambda2_lasso(b, sigma2, tau2, delta2prod)
        elif horseshoe:
            lambda2 = sample_lambda2_hs(b, sigma2, tau2, nu, delta2prod)
            nu = sample_nu_hs(lambda2)
        elif horseshoeplus:
            lambda2 = sample_lambda2_hs(b, sigma2, tau2, nu, delta2prod*phi2)
            nu = sample_nu_hs(lambda2)
            
            phi2 = sample_lambda2_hs(b, sigma2, tau2, zeta, delta2prod*lambda2)
            zeta = sample_nu_hs(phi2)
            
            lambda2 = lambda2 * phi2
            
        # collect samples
        iters += 1
        if iters > burnin:
            # thining
            if iters%thin == 0:
                k += 1
                # store posterior samples
                beta0[0][k] = b0
                beta[:,k] = b.T
                
                # store posterior means
                retval["muB"] += muB
                retval["muB0"] += muB0
                retval["tau2"][:,k] = tau2
                
                # negloglikelihood of the model
                if mu is not None:
                    negll[0][k], lprob, prob = br_regnlike_mu(model, mu, e, y, sigma2, tdof)
                else:
                    negll[0][k] = (nx/2)*np.log(2*np.pi*sigma2) + ete/2/sigma2
                    prob = 0
                    lprob = 0
                
                # calculate WAIC
                waicProb = waicProb + prob
                waicLProb = waicLProb + lprob
                waicLProb2 = waicLProb2 + lprob**2
                
                if not binomial:
                    retval['sigma2'][0][k] = sigma2
                    retval['muSigma2'] = retval['muSigma2'] + muSigma2
                    
                if not (ridge or gprior):
                    retval['lambda2'][:,k] = lambda2.reshape(-1)
                    
    # compute average posterior means
    retval["muB"] /= nsamples
    retval["muB0"] /= nsamples
    if not binomial:
        retval['muSigma2'] /= nsamples
        
    # other stats
    retval['tStats'] = retval['muB']/np.std(beta, axis=1).reshape(-1,1)
    retval['varranks'] = np.array([[None] for i in range(px+1)])

    # if required, compute variable ranking
    if runBFR:
        retval['varranks'] = bfr(beta)
        
    # compute model fit stats
    retval['modelstats'] = br_compute_model_stats(y, X, retval)
    retval['modelstats']['negll'] = negll
    if waic:
        retval['modelstats']['waic_dof'] = np.sum(waicLProb2/nsamples) - np.sum((waicLProb/nsamples)**2)
        retval['modelstats']['waic'] = -np.sum(np.log(waicLProb/nsamples)) + retval['modelstats']['waic_dof']
    else:
        retval['modelstats']['waic_dof'] = np.Inf
        retval['modelstats']['waic'] = np.Inf
        
    # rescale coefficients
    if normalize:
        beta = beta/normX.reshape(-1,1)
        beta0 = beta0 - np.matmul(muX, beta)
        
        retval['muB'] = retval['muB']/normX.reshape(-1,1)
        retval['muB0'] = retval['muB0'] - np.matmul(muX, retval['muB'])
    
    # posterior median estimation
    retval['medB'] = np.median(beta, axis=1)
    retval['medB0'] = np.median(beta0)
    
    retval['runstats']['rundate'] = 0
    retval['runstats']['runtime'] = 0
    
    if display:
        br_summary(beta, beta0, retval)
        
    return retval, beta0, beta

In [33]:
bayesreg(X, y, 'gaussian', 'lasso')

Bayesian linear lasso regression 


Number of obs   =442
Number of vars  =10 

MCMC Samples    =1000
MCMC Burnin     =1000
MCMC Thining    =5 

Root MSE        =[[54.41594431]]
R-Squared       =0.5149562104634555
WAIC            =-733.8845878136796 

------------- + -----------------------------------------------------------------------------------
  Parameter      mean(Coef)    std(Coef)        [95% Cred. Interval]      tStat    Rank       ESS
------------- + -----------------------------------------------------------------------------------
            |    [-3.3245315]    53.81690943566575    [-103.38573338]    [103.08872074]    [-0.06177485]    [10]
            |    [-211.09714209]    61.8530638061265    [-330.32050527]    [-88.20776559]    [-3.41288093]    [4]
            |    [523.3144478]    68.08397267469556    [397.82527793]    [664.05367046]    [7.68630894]    [1]
            |    [305.16971077]    66.60038026537903    [178.99329237]    [436.89156829]    [4.58210163]    [3]
 

C:\Users\Yanda\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:113: RuntimeWarning: overflow encountered in exp
C:\Users\Yanda\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in subtract


({'sparsify_method': '',
  'sparseB0': None,
  'sparseB': None,
  'medB0': 152.06332216663884,
  'medB': array([  -1.43638162, -212.42738847,  525.00438886,  304.79070195,
         -169.85033205,   -2.56808046, -148.36534942,   86.75023213,
          518.46526323,   60.98604438]),
  'muB0': array([152.13348416]),
  'muB': array([[  -3.3245315 ],
         [-211.09714209],
         [ 523.3144478 ],
         [ 305.16971077],
         [-179.69259245],
         [   3.72392847],
         [-154.63838076],
         [  96.3585524 ],
         [ 521.12684798],
         [  63.43768617]]),
  'tau2': array([[ 23.49234215,  76.54482777,  18.43153685,  20.20984248,
           56.47881225,  30.41964691,  22.69885623, 298.25707083,
           33.37806898,  42.88423885,  30.43170778,  39.54118068,
           23.60026341,  16.71942405,  38.13153454,  63.68263968,
           23.66254103,  42.68744284,  20.24397307,  14.47896708,
           31.87122057,  24.81857448,  12.88772055,  36.21701015,
           1

def sample_delta2_hs(b, sigma2, tau2, lambda2, rho, delta2prod, delta2, groups, nGroups, GroupSizes):
    delta2prod = delta2prod/delta2[groups]
    K = b**2/lambda2/delta2prod
    
    scale = np.zeros(shape = (1, nGroups))
    for i in range(nGroups):
        ix = (groups == i)
        scale[i] = 1/rho[i] + 1/2/tau2/sigma2 * np.sum(K[ix], axis=0)
    delta2[:-1] = scale/np.random.gamma(shape = 1, scale=(GroupSizes.T+1)/2)
    
    delta2prod = delta2prod * delta2[groups]
    return delta2, delta2prod

def sample_delta2_lasso(b, sigma2, tau2, lambda2, delta2prod, delta2, groups, nGroups, GroupSizes):
    delta2prod = delta2prod/delta2[groups]
    K = b**2/lambda2/delta2prod
    
    for i in range(nGroups):
        ix = (groups == i)
        
        gig_p = GroupSizes[i]/2 - 1
        gig_a = 1/tau2/sigma2 * np.sum(K[ix], axis=0)
        gig_b = 2
        
        delta2[i] = gigrnd(gig_p, gig_a, gig_b, 1)
    delta2prod = delta2prod * delta2[groups]
    return delta2, delta2prod

def sample_nu_hsplus(lambda2, phi2):
    scale = 1/phi2 + 1/lambda2
    nu = 1/exprnd_fast(1/scale)
    return nu

def sample_omega2_laplace(e, sigma2):
    mu = np.sqrt(2*sigma2/e**2)
    lamda = 2
    omega2 = 1/randinvg(mu, 1/lamda)
    return omega2

def sample_omega2_tdist(e, sigma2, tdof):
    n = len(e)
    a = (tdof + 1)/2
    b = e**2/sigma2/2 + tdof/2
    omega2 = b/np.random.gamma(shape = (n, 1), scale = a)
    return omega2

def sample_phi2_hsplus(mu, zeta):
    scale = 1/nu + 1/zeta
    phi2 = 1/exprnd_fast(1/scale)
    return phi2

def sample_zeta_hsplus(phi2):
    scale = 1 + 1/phi2
    zeta = 1/exprnd_fast(1/scale)
    return zeta